In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path

In [2]:
# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# Custom Dataset class
class HairDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with train/test folders containing curly/straight subfolders
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Load curly hair images (label = 1)
        curly_dir = self.root_dir / 'curly'
        if curly_dir.exists():
            for img_path in curly_dir.glob('*.jpg'):
                self.images.append(str(img_path))
                self.labels.append(1)
            for img_path in curly_dir.glob('*.png'):
                self.images.append(str(img_path))
                self.labels.append(1)
        
        # Load straight hair images (label = 0)
        straight_dir = self.root_dir / 'straight'
        if straight_dir.exists():
            for img_path in straight_dir.glob('*.jpg'):
                self.images.append(str(img_path))
                self.labels.append(0)
            for img_path in straight_dir.glob('*.png'):
                self.images.append(str(img_path))
                self.labels.append(0)
        
        print(f"Loaded {len(self.images)} images from {root_dir}")
        print(f"Curly: {sum(self.labels)}, Straight: {len(self.labels) - sum(self.labels)}")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.float32)

In [4]:
# Define the CNN Model
class HairClassifierCNN(nn.Module):
    def __init__(self):
        super(HairClassifierCNN, self).__init__()
        
        # Convolutional layer: 32 filters, kernel 3x3, padding=0, stride=1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                               kernel_size=(3, 3), padding=0, stride=1)
        self.relu1 = nn.ReLU()
        
        # Max pooling layer: 2x2
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        
        # Calculate flattened size after conv and pooling
        # Input: (3, 200, 200)
        # After Conv2d: (32, 198, 198) because (200 - 3 + 0)/1 + 1 = 198
        # After MaxPool2d: (32, 99, 99) because 198/2 = 99
        self.flatten_size = 32 * 99 * 99
        
        # Fully connected layers
        self.fc1 = nn.Linear(self.flatten_size, 64)
        self.relu2 = nn.ReLU()
        
        # Output layer (no activation - BCEWithLogitsLoss includes sigmoid)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        # Conv + ReLU + MaxPool
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu2(x)
        
        # Output layer
        x = self.fc2(x)
        
        return x

In [5]:
# Data transforms with ImageNet normalization
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )  # ImageNet normalization
])

In [6]:
# Data transforms with augmentation for second training phase
train_transforms_augmented = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )  # ImageNet normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )  # ImageNet normalization
])

In [7]:
# Load datasets
train_dataset = HairDataset('data/train', transform=train_transforms)
test_dataset = HairDataset('data/test', transform=test_transforms)

Loaded 759 images from data/train
Curly: 394, Straight: 365
Loaded 193 images from data/test
Curly: 100, Straight: 93


In [8]:
# Create data loaders with batch_size=20
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [9]:
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [10]:
model = HairClassifierCNN().to(device)
print(model)

HairClassifierCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (relu2): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [11]:
# Count total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


Total parameters: 20,073,473
Trainable parameters: 20,073,473


In [12]:
# Detailed parameter breakdown
print("\nParameter breakdown:")
for name, param in model.named_parameters():
    print(f"{name}: {param.numel():,} parameters")


Parameter breakdown:
conv1.weight: 864 parameters
conv1.bias: 32 parameters
fc1.weight: 20,072,448 parameters
fc1.bias: 64 parameters
fc2.weight: 64 parameters
fc2.bias: 1 parameters


In [13]:
# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy with Logits for binary classification
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [14]:
# Training function with history tracking
def train_model_with_history(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)  # Ensure labels are float and have shape (batch_size, 1)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = correct_train / total_train
        history['loss'].append(epoch_loss)
        history['acc'].append(epoch_acc)
        
        # Validation phase
        model.eval()
        val_running_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels = labels.float().unsqueeze(1)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * images.size(0)
                predicted = (torch.sigmoid(outputs) > 0.5).float()
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
        
        val_epoch_loss = val_running_loss / len(test_dataset)
        val_epoch_acc = correct_val / total_val
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_epoch_acc)
        
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")
    
    return history

In [15]:
# Train the model
print("\nStarting training...")
history = train_model_with_history(model, train_loader, test_loader, criterion, optimizer, num_epochs=10)


Starting training...
Epoch 1/10, Loss: 0.6398, Acc: 0.6324, Val Loss: 0.7477, Val Acc: 0.5751
Epoch 2/10, Loss: 0.5635, Acc: 0.7049, Val Loss: 0.6436, Val Acc: 0.6425
Epoch 3/10, Loss: 0.5038, Acc: 0.7497, Val Loss: 0.6201, Val Acc: 0.6528
Epoch 4/10, Loss: 0.4185, Acc: 0.8076, Val Loss: 0.5979, Val Acc: 0.7098
Epoch 5/10, Loss: 0.4140, Acc: 0.8103, Val Loss: 0.7306, Val Acc: 0.6269
Epoch 6/10, Loss: 0.3764, Acc: 0.8063, Val Loss: 0.7482, Val Acc: 0.6218
Epoch 7/10, Loss: 0.2749, Acc: 0.8893, Val Loss: 0.6358, Val Acc: 0.7306
Epoch 8/10, Loss: 0.2098, Acc: 0.9144, Val Loss: 0.7522, Val Acc: 0.7150
Epoch 9/10, Loss: 0.1741, Acc: 0.9354, Val Loss: 0.7580, Val Acc: 0.6891
Epoch 10/10, Loss: 0.1460, Acc: 0.9486, Val Loss: 0.9511, Val Acc: 0.6632


In [16]:
# Calculate statistics for Questions 3 and 4
print("\n" + "="*60)
print("ANSWERS TO HOMEWORK QUESTIONS (First 10 epochs)")
print("="*60)


ANSWERS TO HOMEWORK QUESTIONS (First 10 epochs)


In [17]:
# Question 3: Median of training accuracy
median_train_acc = np.median(history['acc'])
print(f"\nQuestion 3 - Median of training accuracy: {median_train_acc:.4f}")
print(f"Training accuracies: {[f'{acc:.4f}' for acc in history['acc']]}")


Question 3 - Median of training accuracy: 0.8090
Training accuracies: ['0.6324', '0.7049', '0.7497', '0.8076', '0.8103', '0.8063', '0.8893', '0.9144', '0.9354', '0.9486']


In [18]:
# Question 4: Standard deviation of training loss
std_train_loss = np.std(history['loss'])
print(f"\nQuestion 4 - Standard deviation of training loss: {std_train_loss:.4f}")
print(f"Training losses: {[f'{loss:.4f}' for loss in history['loss']]}")

print("\n" + "="*60)


Question 4 - Standard deviation of training loss: 0.1599
Training losses: ['0.6398', '0.5635', '0.5038', '0.4185', '0.4140', '0.3764', '0.2749', '0.2098', '0.1741', '0.1460']



In [19]:
# ============================================================================
# CONTINUE TRAINING WITH DATA AUGMENTATION (Questions 5 and 6)
# ============================================================================

print("\n" + "="*60)
print("CONTINUING TRAINING WITH DATA AUGMENTATION")
print("="*60)


CONTINUING TRAINING WITH DATA AUGMENTATION


In [20]:
# Create new training dataset with augmentation
train_dataset_augmented = HairDataset('data/train', transform=train_transforms_augmented)
train_loader_augmented = DataLoader(train_dataset_augmented, batch_size=20, shuffle=True)

Loaded 759 images from data/train
Curly: 394, Straight: 365


In [21]:
# Continue training for 10 more epochs (DO NOT recreate the model)
print("\nContinuing training with augmented data for 10 more epochs...")
history_augmented = train_model_with_history(model, train_loader_augmented, test_loader, 
                                             criterion, optimizer, num_epochs=10)


Continuing training with augmented data for 10 more epochs...
Epoch 1/10, Loss: 0.6718, Acc: 0.6034, Val Loss: 0.7322, Val Acc: 0.6632
Epoch 2/10, Loss: 0.5742, Acc: 0.6825, Val Loss: 0.7526, Val Acc: 0.6684
Epoch 3/10, Loss: 0.5535, Acc: 0.6877, Val Loss: 0.7080, Val Acc: 0.6425
Epoch 4/10, Loss: 0.5327, Acc: 0.7141, Val Loss: 0.6229, Val Acc: 0.7150
Epoch 5/10, Loss: 0.5155, Acc: 0.7339, Val Loss: 0.6375, Val Acc: 0.6891
Epoch 6/10, Loss: 0.4970, Acc: 0.7510, Val Loss: 0.6838, Val Acc: 0.6788
Epoch 7/10, Loss: 0.4587, Acc: 0.7839, Val Loss: 0.6653, Val Acc: 0.6684
Epoch 8/10, Loss: 0.4769, Acc: 0.7655, Val Loss: 0.5778, Val Acc: 0.7306
Epoch 9/10, Loss: 0.4639, Acc: 0.7615, Val Loss: 0.7227, Val Acc: 0.6839
Epoch 10/10, Loss: 0.4728, Acc: 0.7589, Val Loss: 0.5665, Val Acc: 0.7098


In [22]:
# Calculate statistics for Questions 5 and 6
print("\n" + "="*60)
print("ANSWERS TO HOMEWORK QUESTIONS (Augmented training)")
print("="*60)


ANSWERS TO HOMEWORK QUESTIONS (Augmented training)


In [23]:
# Question 5: Mean of test loss for all epochs
mean_test_loss = np.mean(history_augmented['val_loss'])
print(f"\nQuestion 5 - Mean of test loss (all 10 epochs): {mean_test_loss:.4f}")
print(f"Test losses: {[f'{loss:.4f}' for loss in history_augmented['val_loss']]}")


Question 5 - Mean of test loss (all 10 epochs): 0.6669
Test losses: ['0.7322', '0.7526', '0.7080', '0.6229', '0.6375', '0.6838', '0.6653', '0.5778', '0.7227', '0.5665']


In [24]:
# Question 6: Average of test accuracy for last 5 epochs (epochs 6-10)
avg_test_acc_last5 = np.mean(history_augmented['val_acc'][5:10])
print(f"\nQuestion 6 - Average of test accuracy (epochs 6-10): {avg_test_acc_last5:.4f}")
print(f"Test accuracies (last 5 epochs): {[f'{acc:.4f}' for acc in history_augmented['val_acc'][5:10]]}")
print(f"All test accuracies: {[f'{acc:.4f}' for acc in history_augmented['val_acc']]}")

print("\n" + "="*60)


Question 6 - Average of test accuracy (epochs 6-10): 0.6943
Test accuracies (last 5 epochs): ['0.6788', '0.6684', '0.7306', '0.6839', '0.7098']
All test accuracies: ['0.6632', '0.6684', '0.6425', '0.7150', '0.6891', '0.6788', '0.6684', '0.7306', '0.6839', '0.7098']



In [25]:
# Final evaluation on test set
print("\nFinal evaluation on test set...")
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1)
        
        outputs = model(images)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

final_accuracy = 100 * correct / total
print(f'Final Test Accuracy: {final_accuracy:.2f}%')


Final evaluation on test set...
Final Test Accuracy: 70.98%


In [26]:
# Save the model (optional)
torch.save(model.state_dict(), 'hair_classifier_model.pth')
print("\nModel saved to 'hair_classifier_model.pth'")


Model saved to 'hair_classifier_model.pth'
